In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('hotels_data.csv')

In [3]:
df.head()

,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars
0,1,7/17/2015 0:00,8/12/2015 0:00,5,1178,1040,1,6,Best Western Plus Seaport Inn Downtown,3
1,1,7/17/2015 0:00,8/19/2015 0:00,5,1113,982,1,8,Best Western Plus Seaport Inn Downtown,3
2,1,7/17/2015 0:00,8/13/2015 0:00,5,4370,4240,1,3,The Peninsula New York,5
3,1,7/17/2015 0:00,7/26/2015 0:00,5,1739,1667,1,18,Eventi Hotel a Kimpton Hotel,4
4,1,7/17/2015 0:00,8/12/2015 0:00,5,1739,1672,1,3,Eventi Hotel a Kimpton Hotel,4


In [17]:
# get the 150 hotels with maximum records
groupedCountTop = df.groupby(['Hotel Name']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(150)

In [19]:
groupedCountTop.head()

,Hotel Name,counts
327,Newark Liberty International Airport Marriott,5346
194,Hilton Garden Inn Times Square,4892
374,Residence Inn Newark Elizabeth Liberty Interna...,4314
538,Westin New York at Times Square,3792
290,Loews Regency New York Hotel,3617


In [28]:
df = df[df['Hotel Name'].isin(groupedCountTop['Hotel Name'])]

In [30]:
# get the 40 checkin dates with maximum records
groupedCountTop = df.groupby(['Checkin Date']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(40)

In [31]:
groupedCountTop.head()

,Checkin Date,counts
64,11/11/2015 0:00,2302
36,10/14/2015 0:00,1887
86,11/4/2015 0:00,1885
151,8/19/2015 0:00,1883
51,10/28/2015 0:00,1861


In [32]:
df = df[df['Checkin Date'].isin(groupedCountTop['Checkin Date'])]